## ECB cut-and-paste
Write a k=v parsing routine, as if for a structured cookie. The routine should take:
```
foo=bar&baz=qux&zap=zazzle
```
... and produce:
```
{
  foo: 'bar',
  baz: 'qux',
  zap: 'zazzle'
}
```
(you know, the object; I don't care if you convert it to JSON).

Now write a function that encodes a user profile in that format, given an email address. You should have something like:
```
profile_for("foo@bar.com")
```
... and it should produce:
```
{
  email: 'foo@bar.com',
  uid: 10,
  role: 'user'
}
```
... encoded as:
```
email=foo@bar.com&uid=10&role=user
```
Your "profile_for" function should not allow encoding metacharacters (& and =). Eat them, quote them, whatever you want to do, but don't let people set their email address to "foo@bar.com&role=admin".

Now, two more easy functions. Generate a random AES key, then:

A. Encrypt the encoded user profile under the key; "provide" that to the "attacker". <br>
B. Decrypt the encoded user profile and parse it.
Using only the user input to profile_for() (as an oracle to generate "valid" ciphertexts) and the ciphertexts themselves, make a role=admin profile.

In [2]:
import os
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

In [5]:
KEY = os.urandom(16)

def kv_parser(text: str):
    profile = {}
    items = text.split('&')
    for item in items:
        if '=' in item:
            data_k, data_v = item.split('=',1)
            profile[data_k] = data_v
    return profile

# "email=foo@bar.co.id&uid=10&role=user"
def profile_for(email: str):
    filter = "!#$%^&*(+=|/><,'~)"
    for char in email:
        if char in filter:
            email = email.replace(char,"")
    
    return ''.join(f"email={email}&uid={10}&role=user")

def encrypt_profile(email: str):
    encoded_profile = profile_for(email)
    data_b = encoded_profile.encode()

    cipher = AES.new(KEY,AES.MODE_ECB)
    return cipher.encrypt(pad(data_b,16))

def decrypt_profile(cipher_data:bytes):
    cipher = AES.new(KEY,AES.MODE_ECB)
    data_d = cipher.decrypt(cipher_data)

    try:
        unpadded = unpad(data_d,16)
        return kv_parser(unpadded.decode())
    except ValueError:
        return

# normal use
test = "foo@bar.com"
e_profile = encrypt_profile(test)
d_profile = decrypt_profile(e_profile)

# hacker use
def pkcs7_pad(data: bytes, block_size: int = 16) -> bytes:
    remainder = len(data) % block_size
    padding_len = block_size - remainder
    padding = bytes([padding_len]) * padding_len
    return data + padding

# step1
padd_admin = pkcs7_pad(b'admin')
payloads1 = 'A'*10 + padd_admin.decode('utf-8')

p1_result_e = encrypt_profile(payloads1)
p1_result_d = decrypt_profile(p1_result_e)
print("Hasil encrypt (16 - 32 byte): ",p1_result_e[16:32])
print("Hasil object : ",p1_result_d)

# step2
payloads2 = "foo@bar.co.id"
p2_result_e = encrypt_profile(payloads2)
p2_result_d = decrypt_profile(p2_result_e)
print("Hasil encrypt (32 byte after): ",p2_result_e[:32])
print("Hasil object : ",p2_result_d)

# step3
append_payloads = p2_result_e[:32] + p1_result_e[16:32]
result = decrypt_profile(append_payloads)
print(result)

Hasil encrypt (16 - 32 byte):  b'\xec\xef9:\x0c\x9e\x15+\xe0*\x80\x05g\xd4\x85m'
Hasil object :  {'email': 'AAAAAAAAAAadmin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b', 'uid': '10', 'role': 'user'}
Hasil encrypt (32 byte after):  b'V\x0fI\x1c\xe8c\xca\x92~\xf7\x04\xe8\xbb~\xef\xc2\x8335\x97\xef\x90\xff\x05\xb0\x9f\xf3\xf9\xcaL\xd1e'
Hasil object :  {'email': 'foo@bar.co.id', 'uid': '10', 'role': 'user'}
{'email': 'foo@bar.co.id', 'uid': '10', 'role': 'admin'}
